# 1. Import modules & Read files

Import required modules.

In [1]:
import re
import numpy as np
import Levenshtein as levenshtein
import json
import fuzzy
from sklearn.model_selection import train_test_split
import soundex

Define a function to read the files.

In [2]:
def readfile(filename):
    f = open(filename, "r")
    lines = f.readlines()
    ls = []
    for line in lines:
        ls.append(line.strip())
    f.close()
    return ls

Read dicitonary and wiki dataset and store as variables.

In [3]:
dict_ls = readfile("dict.txt")
correct_ls = readfile("wiki_correct.txt")
misspell_ls = readfile("wiki_misspell.txt")

# 2. Evaluation Metrics

Define a function to evaluate the spelling correction results. (Precision & Recall & F-Score)

In [4]:
def wiki_eval(result_dict, curr_misspell_ls, curr_correct_ls):
    tp = 0
    fp = 0
    fn = 0

    for i in range(len(curr_misspell_ls)):
        word = curr_misspell_ls[i]
        true_word = curr_correct_ls[i]
        if true_word in result_dict[word]:
            tp += 1
            fp += len(result_dict[word]) - 1
        else:
            fn += 1 # miss
            fp += len(result_dict[word])
            
    precision = tp*1.0 / (tp + fp)
    recall = tp*1.0 / (tp + fn)
    fscore = 2*(precision*recall)/(precision + recall)

    return precision,recall,fscore

# 3. Baseline method (Levenshtein Distance)
## 3.1. Run with the whole Wikipedia Dataset (4453 tokens)

Find the best matches for the word in dictionary according to the levenshtein distance (ged).

In [23]:
def best_match_levenshtein(target):
    # init best match as first entry of the dict
    min_dist = levenshtein.distance(target,dict_ls[0])
    best_matches = [dict_ls[0]]
    
    for word in dict_ls[1:]:
        if abs(len(word) - len(target)) > min_dist:  # not possible to be min_dist, skip
            continue
        dist = levenshtein.distance(target,word)  # cal global edit distance
        # replace if shorter distance
        if dist < min_dist:
            min_dist = dist
            best_matches = [word]
        elif dist == min_dist:
            best_matches.append(word)
    
    return best_matches

Find the best matches for the words in wiki data sets according to levenshtein.

In [ ]:
levenshtein_correction_dict = {}
for word in wiki_misspell_ls:
    if word not in levenshtein_correction_dict: # avoid repeated word
        levenshtein_correction_dict[word] = best_match_levenshtein(word)

Save the correction results to json file.

In [ ]:
with open('levenshtein_correction.json', 'w') as fp:
    json.dump(levenshtein_correction_dict, fp)
    fp.close()

Load the results from json file if required.|

In [6]:
with open('levenshtein_correction.json', 'r') as fp:
    levenshtein_correction_dict = json.load(fp) 
    fp.close()

Precision, recall, and F-score for Levenshtein (baseline)

In [16]:
(precision, recall, fscore) = wiki_eval(levenshtein_correction_dict, misspell_ls, correct_ls)
print "Evaluation Metrics (Levenshtein Distance, 100% Data)"
print "---------------------------------------------------"
print "Precision: " + str(precision)
print "Recall: " + str(recall)
print "F-score: " + str(precision)

Evaluation Metrics (Levenshtein Distance, 100% Data)
---------------------------------------------------
Precision: 0.260432080497
Recall: 0.790478329216
F-score: 0.260432080497


## 3.2. Run with only 20% of the Wikipedia Dataset

### 3.2.1. Randomly extract 20% of the Wikipedia Dataset as a subset

Randomly extract the subset with "train_test_split" function.

In [46]:
misspell_subset, misspell_left, correct_subset, correct_left = train_test_split(misspell_ls, 
                                                                                correct_ls,
                                                                                train_size = 0.2,
                                                                                test_size = 0.8,
                                                                                shuffle = True)

Combine two subsets into a single list.

In [47]:
combined_subset = [(misspell_subset), (correct_subset)]

Save the combined subset to numpy file.

In [48]:
# Save
np.save('combined_subset.npy', combined_subset) 

Load the combined subset from numpy file and store as variables if required.

In [49]:
# Load
combined_subset = np.load('combined_subset.npy')

# store as variables
misspell_subset = combined_subset[0]
correct_subset = combined_subset[1]

### 3.2.2. Run Levenshtein with the data subset

Find the best matches for the words in data subset according to Levenshtein.

In [50]:
levenshtein_subcorrection_dict = {}
for word in misspell_subset:
    if word not in levenshtein_subcorrection_dict: # avoid repeated word
        levenshtein_subcorrection_dict[word] = best_match_levenshtein(word)

Save correction correction results.

In [51]:
# Save
np.save('subset_levenshtein_results.npy', levenshtein_subcorrection_dict)

In [52]:
# Load
levenshtein_subcorrection_dict = np.load('subset_levenshtein_results.npy').item()

Precision, recall, and F-score for Levenshtein (baseline)

In [56]:
(precision, recall, fscore) = wiki_eval(levenshtein_subcorrection_dict, misspell_subset, correct_subset)
print "Evaluation Metrics (Levenshtein Distance, 20% Data)"
print "---------------------------------------------------"
print "Precision: " + str(precision)
print "Recall: " + str(recall)
print "F-score: " + str(precision)

Evaluation Metrics (Levenshtein Distance, 20% Data)
---------------------------------------------------
Precision: 0.269969278034
Recall: 0.789887640449
F-score: 0.269969278034


# 4. Global Edit Distance (with customized parameters)

GED implementation with customized parameter set

In [57]:
# transform s1 (columns) to s2 (rows)
def edit_distance(s1,s2, params):
    (m,i,d,r) = params
    
    # init matrix
    s1_len = len(s1)
    s2_len = len(s2)
    A = np.zeros((s2_len+1,s1_len+1))
    A[0][0] = 0
    for j in range(1, s2_len+1):
        A[j][0] = j*i;  # insert
    for k in range(1, s1_len+1):
        A[0][k] = k*d;  # delete
        
    # filling in table
    for j in range(1,s2_len+1):
        for k in range(1,s1_len+1):
            A[j][k] = max(A[j][k-1] + d,  # delete
                          A[j-1][k] + i,  # insert
                          A[j-1][k-1] + equal(s1[k-1],s2[j-1],m,r))  # match/replace
    return A[s2_len,s1_len]
    
def equal(ch1,ch2,cost1,cost2):
    if ch1 == ch2:
        return cost1
    else:
        return cost2

Testing implementation

In [58]:
edit_distance("crat","arts",(1,-1,-1,-1))

-1.0

## 4.1. Parameter set ()

# 5. N-Gram Distance

# 6. Soundex

Find the best match(es) for the word in dictionary according to the soundex table.

In [ ]:
soundex = fuzzy.Soundex(4)
soundex("fuzzy")

In [ ]:
def best_match_soundex(target):
    
    soundex = fuzzy.Soundex(4)
    target_soundex = soundex(target)
    
    best_matches = []
    
    for word in dict_ls:
        print word
        print soundex(word)
        if (soundex(word) == target_soundex):  # same soundex
            best_matches.append(word)
    print best_matches
    
    return best_matches

In [ ]:
wiki_correction_dict_soundex = {}
for word in wiki_misspell_ls:
    if word not in wiki_correction_dict_soundex:
        wiki_correction_dict_soundex[word] = best_match_soundex(word)

In [ ]:
soundex = fuzzy.Soundex(4)
soundex("aam")

In [ ]:
print wiki_dict_ls